# EEL891 - 2024.2 - Trabalho 2
### Regressão Multivariável - Estimar Preços de Imóveis
*Aluno:* Miguel Badany Cerne<br>*DRE:* 123370433<br>*Usuário Kaggle:* miguelbadanycerne<br>*ID Kaggle:* 20164449
<hr>

# 1. Baixar e Importar Bibliotecas

In [119]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 2. Exploração e Tratamento de Dados

In [120]:
df_treino = pd.read_csv('./conjunto_de_treinamento.csv')
df_treino.drop(['Id'], axis=1, inplace=True)
df_teste = pd.read_csv('./conjunto_de_teste.csv')
df_teste.drop(['Id'], axis=1, inplace=True)

In [121]:
df_treino.head(3).T

,0,1,2
tipo,Casa,Apartamento,Apartamento
bairro,Imbiribeira,Casa Amarela,Encruzilhada
tipo_vendedor,Imobiliaria,Imobiliaria,Imobiliaria
quartos,3,4,3
suites,3,4,1
vagas,5,2,0
area_util,223,157,53
area_extra,167,0,0
diferenciais,piscina e copa,piscina e churrasqueira,nenhum
churrasqueira,0,1,0


In [122]:
df_treino.columns

Index(['tipo', 'bairro', 'tipo_vendedor', 'quartos', 'suites', 'vagas',
       'area_util', 'area_extra', 'diferenciais', 'churrasqueira',
       'estacionamento', 'piscina', 'playground', 'quadra', 's_festas',
       's_jogos', 's_ginastica', 'sauna', 'vista_mar', 'preco'],
      dtype='object')

Notamos que as seguintes features não são numéricas:
- tipo
- bairro
- tipo_vendedor
- diferenciais<br>
Devemos tratar melhor cada uma dessas colunas posteriormente.

In [123]:
df_treino['diferenciais'].unique()

array(['piscina e copa', 'piscina e churrasqueira', 'nenhum',
       'playground', 'piscina e playground', 'sauna e playground',
       'piscina e sauna', 'piscina e quadra poliesportiva',
       'salao de festas', 'campo de futebol e playground',
       'piscina e estacionamento visitantes', 'copa e salao de festas',
       'esquina e salao de festas', 'piscina e salao de festas',
       'playground e sala de ginastica', 'copa e playground',
       'estacionamento visitantes e playground',
       'playground e salao de festas', 'churrasqueira e playground',
       'churrasqueira e copa', 'churrasqueira e salao de festas', 'copa',
       'esquina', 'piscina', 'playground e quadra poliesportiva',
       'estacionamento visitantes', 'piscina e esquina', 'copa e esquina',
       'estacionamento visitantes e salao de festas',
       'piscina e campo de futebol',
       'frente para o mar e salao de festas', 'churrasqueira',
       'churrasqueira e campo de futebol', 'piscina e frente para 

Para viabilizar o One-Hot Encoding para esta coluna, faremos um mapeamento dessas classes para diminuir o número total de classes.
1. Simplificamos as categorias, aglutinando todas combinações de mais de um elemento no nome do primeiro elemento, seguido de '++'
2. Anulamos os valores 'nenhum', 'churrasqueira', 'estacionamento'... haja vista que, no conjunto de dados, temos colunas exclusivas para estes valores

In [124]:
valores_diferenciais = set(df_treino['diferenciais'].unique().tolist() + df_teste['diferenciais'].unique().tolist())

diferenciais = {valor: valor for valor in valores_diferenciais}

for valor in valores_diferenciais:
    if 'futebol e' in valor:
        diferenciais[valor] = 'futebol++'
    if 'care e' in valor:
        diferenciais[valor] = 'children care++'
    if 'churrasqueira e' in valor:
        diferenciais[valor] = 'churrasqueira++'
    if 'copa e' in valor:
        diferenciais[valor] = 'copa++'
    if 'esquina e' in valor:
        diferenciais[valor] = 'esquina++'
    if 'estacionamento visitantes e' in valor:
        diferenciais[valor] = 'estacionamento visita++'
    if 'mar e' in valor:
        diferenciais[valor] = 'mar++'
    if 'piscina e' in valor:
        diferenciais[valor] = 'piscina++'
    if 'playground e' in valor:
        diferenciais[valor] = 'playground++'
    if 'quadra poliesportiva e' in valor:
        diferenciais[valor] = 'quadra++'
    if 'ginastica e' in valor:
        diferenciais[valor] = 'ginastica++'
    if 'festas e' in valor:
        diferenciais[valor] = 'festas++'
    if 'jogos e' in valor:
        diferenciais[valor] = 'festas++'
    if 'sauna e' in valor:
        diferenciais[valor] = 'sauna++'
    if 'hidromassagem e' in valor:
        diferenciais[valor] = 'hidromassagem++'
    if valor == 'churrasqueira':
        diferenciais[valor] = np.nan
    if valor == 'estacionamento':
        diferenciais[valor] = np.nan
    if valor == 'piscina':
        diferenciais[valor] = np.nan
    if valor == 'playground':
        diferenciais[valor] = np.nan
    if valor == 'quadra poliesportiva':
        diferenciais[valor] = np.nan
    if valor == 'salao de festas':
        diferenciais[valor] = np.nan
    if valor == 'salao de jogos':
        diferenciais[valor] = np.nan
    if valor == 'sauna':
        diferenciais[valor] = np.nan
    if valor == 'frente para o mar':
        diferenciais[valor] = np.nan
    if valor == 'nenhum':
        diferenciais[valor] = np.nan
    

diferenciais

{'frente para o mar e esquina': 'mar++',
 'campo de futebol e estacionamento visitantes': 'futebol++',
 'piscina e campo de futebol': 'piscina++',
 'piscina e playground': 'piscina++',
 'frente para o mar': nan,
 'campo de futebol e esquina': 'futebol++',
 'salao de festas e vestiario': 'festas++',
 'piscina e churrasqueira': 'piscina++',
 'piscina e children care': 'piscina++',
 'piscina e copa': 'piscina++',
 'hidromassagem e salao de festas': 'hidromassagem++',
 'frente para o mar e copa': 'mar++',
 'copa': 'copa',
 'esquina e salao de festas': 'esquina++',
 'estacionamento visitantes e salao de jogos': 'estacionamento visita++',
 'campo de futebol e copa': 'futebol++',
 'estacionamento visitantes e playground': 'estacionamento visita++',
 'campo de futebol e quadra poliesportiva': 'futebol++',
 'churrasqueira e copa': 'churrasqueira++',
 'playground e quadra poliesportiva': 'playground++',
 'churrasqueira e children care': 'churrasqueira++',
 'sauna e copa': 'sauna++',
 'playground

In [125]:
df_treino = df_treino.replace(diferenciais)
df_teste = df_teste.replace(diferenciais)

In [126]:
df_treino.head(10).T

,0,1,2,3,4,5,6,7,8,9
tipo,Casa,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento
bairro,Imbiribeira,Casa Amarela,Encruzilhada,Boa Viagem,Rosarinho,Boa Vista,Rosarinho,Espinheiro,Boa Viagem,Boa Viagem
tipo_vendedor,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria
quartos,3,4,3,4,2,3,4,3,4,3
suites,3,4,1,3,1,0,2,1,4,1
vagas,5,2,0,2,1,1,0,1,3,1
area_util,223,157,53,149,54,130,105,124,170,72
area_extra,167,0,0,0,0,0,0,0,0,0
diferenciais,piscina++,piscina++,NaN,piscina++,piscina++,NaN,piscina++,NaN,piscina++,NaN
churrasqueira,0,1,0,1,1,0,0,0,1,0


In [127]:
# Checando a coluna 'diferenciais'
df_treino['diferenciais'].unique()

array(['piscina++', nan, 'sauna++', 'futebol++', 'copa++', 'esquina++',
       'playground++', 'estacionamento visita++', 'churrasqueira++',
       'copa', 'esquina', 'estacionamento visitantes', 'mar++',
       'ginastica++', 'festas++', 'sala de ginastica', 'quadra++',
       'children care', 'vestiario', 'children care++'], dtype=object)

In [128]:
df_teste['diferenciais'].unique()

array(['copa++', nan, 'piscina++', 'estacionamento visita++',
       'playground++', 'copa', 'futebol++', 'mar++', 'churrasqueira++',
       'estacionamento visitantes', 'quadra++', 'sauna++', 'festas++',
       'ginastica++', 'hidromassagem++', 'vestiario', 'esquina++',
       'esquina', 'sala de ginastica'], dtype=object)

## 2.1. Checando as colunas categóricas

Com o auxílio do google, conseguimos classificar todos bairros em 3 categorias: baixa (classe baixa), media (classe média) e alta (classe alta). Assim, melhoramos a eficácia do One-Hot Encoding novamente.

In [129]:
bairros = {'Aflitos': 'baixa',
 'Afogados':'baixa',
 'Agua Fria':'baixa',
 'Apipucos':'media',
 'Areias':'media',
 'Arruda':'media',
 'Barro':'baixa',
 'Beberibe':'baixa',
 'Beira Rio':'baixa',
 'Benfica':'baixa',
 'Boa Viagem':'alta',
 'Boa Vista':'media',
 'Bongi':'media',
 'Cajueiro':'media',
 'Campo Grande':'media',
 'Casa Amarela':'media',
 'Casa Forte':'alta',
 'Caxanga':'media',
 'Centro':'media',
 'Cid Universitaria':'media',
 'Coelhos':'baixa',
 'Cohab':'baixa',
 'Cordeiro':'media',
 'Derby':'media',
 'Dois Irmaos':'media',
 'Encruzilhada':'media',
 'Engenho do Meio':'media',
 'Espinheiro':'alta',
 'Estancia':'baixa',
 'Fundao':'baixa',
 'Gracas':'alta',
 'Guabiraba':'baixa',
 'Hipodromo':'baixa',
 'Ibura': 'baixa',
 'Ilha do Leite':'media',
 'Ilha do Retiro':'alta',
 'Imbiribeira':'media',
 'Ipsep':'media',
 'Iputinga':'media',
 'Jaqueira':'alta',
 'Jd S Paulo':'media',
 'Lagoa do Araca':'media',
 'Macaxeira':'baixa',
 'Madalena':'alta',
 'Monteiro':'media',
 'Paissandu':'alta',
 'Parnamirim':'media',
 'Piedade':'baixa',
 'Pina':'media',
 'Poco':'baixa',
 'Poco da Panela':'media',
 'Ponto de Parada':'baixa',
 'Prado':'media',
 'Recife':'media',
 'Rosarinho':'media',
 'S Jose':'alta',
 'San Martin':'media',
 'Sancho':'media',
 'Santana':'media',
 'Setubal':'alta',
 'Soledade':'media',
 'Sto Amaro':'media',
 'Sto Antonio':'media',
 'Tamarineira':'alta',
 'Tejipio':'media',
 'Torre':'media',
 'Torreao':'media',
 'Varzea':'media',
 'Zumbi':'baixa'}

df_treino = df_treino.replace(bairros)
df_teste = df_teste.replace(bairros)

In [130]:
# Checando classificação dos bairros
df_treino['bairro'].unique()

array(['media', 'alta', 'baixa'], dtype=object)

In [131]:
print(df_treino['tipo'].unique())
print(df_treino['tipo_vendedor'].unique())

['Casa' 'Apartamento' 'Quitinete' 'Loft']
['Imobiliaria' 'Pessoa Fisica']


Estudando as duas últimas features categóricas notamos que elas não são problemáticas para a aplicação do One-Hot Encoding. Então, podemos finalizar o tratamento dessas features.

### 2.1.1. Aplicação do One-Hot

In [132]:
df_treino = pd.get_dummies(df_treino, columns=['tipo', 'bairro', 'tipo_vendedor', 'diferenciais'], dtype=int)
df_teste = pd.get_dummies(df_teste, columns=['tipo', 'bairro', 'tipo_vendedor', 'diferenciais'], dtype=int)

In [133]:
print(df_treino.columns.tolist() == df_teste.columns.tolist())
treino_col = df_treino.columns.tolist()
teste_col = df_teste.columns.tolist()
for col in teste_col:
    if col not in treino_col:
        print(col)

print('---')
for col in treino_col:
    if col not in teste_col:
        print(col)

False
diferenciais_hidromassagem++
---
preco
tipo_Quitinete
diferenciais_children care
diferenciais_children care++
